In [4]:
import os
from PIL import Image, ImageEnhance
import numpy as np
from tqdm import tqdm

def apply_mask(image, mask):
    """
    将掩码应用到图片上，根据掩码颜色决定是否覆盖原图
    """
    image_np = np.array(image)
    mask_np = np.array(mask)
    
    # 增加掩码对比度
    enhancer = ImageEnhance.Contrast(mask)
    mask = enhancer.enhance(2.0)
    
    mask_np = np.array(mask)
    
    # 如果掩码为黑色（值为0），则覆盖原图像素
    mask_binary = (mask_np == 0).astype(np.uint8)
    
    # 应用掩码，覆盖黑色区域
    masked_image = image_np * (1 - mask_binary[:, :, np.newaxis]) + np.zeros_like(image_np) * mask_binary[:, :, np.newaxis]
    
    # 确保掩码区域透明处理
    alpha_channel = (1 - mask_binary) * 255
    masked_image = np.dstack((masked_image, alpha_channel))
    
    return Image.fromarray(masked_image.astype(np.uint8))

def merge_images(image_dir, mask_dir, output_dir):
    """
    合并图片和掩码，并保存到新的文件夹
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('.png')]
    
    for image_file in tqdm(image_files, desc="Processing images"):
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, image_file)
        
        if not os.path.exists(mask_path):
            print(f"掩码缺失: {mask_path}")
            continue
        
        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')  # 假设掩码是灰度图
        
        mask_np = np.array(mask)
        
        # 计算黑色像素的比例
        black_pixels_ratio = np.sum(mask_np == 0) / mask_np.size
        
        # 如果黑色像素占比超过94%，跳过这张图片
        if black_pixels_ratio > 0.94:
            # print(f"掩码黑色像素比例过高({black_pixels_ratio*100:.2f}%)，跳过合并: {image_file}")
            continue
        
        masked_image = apply_mask(image, mask)
        output_path = os.path.join(output_dir, image_file)
        masked_image.save(output_path, 'PNG')  # 保存为PNG格式以保留透明度
        # print(f"图片已合并并保存到 {output_path}！")

# 定义图片、掩码和输出文件夹路径
image_dir = r'/data1/dxw_data/llm/redbook_final/data2/all_processed_data'  # 替换为你的正常图片文件夹路径
mask_dir = r'/data1/dxw_data/llm/redbook_final/script_next/data_img_20%_segcloth_background'  # 替换为你的掩码图片文件夹路径
output_dir = r'/data1/dxw_data/llm/redbook_final/script_next/combined_seg_img_pure_094'  # 替换为你的输出文件夹路径

merge_images(image_dir, mask_dir, output_dir)


Processing images:   0%|          | 0/14468 [00:00<?, ?it/s]

Processing images: 100%|██████████| 14468/14468 [26:54<00:00,  8.96it/s]
